In [1]:
import pyfits
import numpy as np
from astropy.coordinates import SkyCoord, get_moon, EarthLocation, ICRS, GCRS,AltAz
from astropy.time import Time
from astropy.io.fits.hdu.hdulist import HDUList
from datetime import date
from astral import Astral, Location
from astropy.time import Time
import astropy.units as u

In [19]:
import pandas as pd
from itertools import chain

In [138]:
## Finding out which file years to download
all_sky=pd.read_csv('objSKY.csv')
all_sky.head()
print('earliest date=', min(all_sky['MJD']), 'or 12-11-2009')
print('latest date=', max(all_sky['MJD']), 'or 05-12-2016')

earliest date= 55176 or 12-11-2009
latest date= 57520 or 05-12-2016


In [139]:
## To check for reading in the right file
rand_files=('spec-3586-55181-0496.fits','spec-3586-55181-0788.fits','spec-3586-55181-0996.fits',
            'spec-3761-55272-0008.fits','spec-10000-57346-0334.fits','spec-3761-55272-0475.fits',
            'spec-10000-57346-0659.fits','spec-5478-56014-0654.fits','spec-10000-57346-0955.fits',
            'spec-5478-56014-0716.fits','spec-10000-57346-0994.fits')

## To look for patterns
inseq_files=('spec-3663-55176-0010.fits','spec-3663-55176-0012.fits','spec-3663-55176-0020.fits',
             'spec-3663-55176-0024.fits','spec-3663-55176-0036.fits','spec-3663-55176-0038.fits',
             'spec-3663-55176-0048.fits','spec-3663-55176-0052.fits','spec-3663-55176-0056.fits',
             'spec-3663-55176-0068.fits','spec-3663-55176-0075.fits','spec-3663-55176-0078.fits',
             'spec-3663-55176-0090.fits','spec-3663-55176-0094.fits','spec-3663-55176-0096.fits',
             'spec-3663-55176-0108.fits','spec-3663-55176-0112.fits','spec-3663-55176-0114.fits',
             'spec-3663-55176-0128.fits','spec-3663-55176-0134.fits')

In [149]:
## We only need year, month, day of month, and mill. of sol. disk
## Cols         1-4    5-6      7-8               31-34
demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open('g2009.txt').readlines()))

In [247]:
## Function to organize mill. of sol. disk by date
def organize_by_date(txt_file):
    ## Pulling out cols of interest
    demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open(txt_file).readlines()))
    ## Organizing cols
    str_list=[demo[i*4:i*4+4] for i in range(int(len(demo)/4))]
    ## Converting to float
    float_dates=[]
    for k in range(len(str_list)):
        float_dates.append([float(i) for i in str_list[k]])
    return float_dates

In [248]:
final_list=organize_by_date('g2009.txt')
final_list

[[2009.0, 1.0, 9.5, 30.0],
 [2009.0, 1.0, 10.5, 51.0],
 [2009.0, 1.0, 11.5, 92.0],
 [2009.0, 1.0, 12.5, 18.0],
 [2009.0, 1.0, 13.5, 17.0],
 [2009.0, 1.0, 19.5, 18.0],
 [2009.0, 2.0, 11.5, 9.0],
 [2009.0, 2.0, 12.5, 13.0],
 [2009.0, 2.0, 13.5, 16.0],
 [2009.0, 2.0, 24.5, 31.0],
 [2009.0, 2.0, 25.5, 16.0],
 [2009.0, 2.0, 26.5, 16.0],
 [2009.0, 3.0, 6.5, 39.0],
 [2009.0, 3.0, 7.5, 38.0],
 [2009.0, 4.0, 21.5, 7.0],
 [2009.0, 4.0, 29.5, 17.0],
 [2009.0, 4.0, 30.5, 3.0],
 [2009.0, 5.0, 13.5, 16.0],
 [2009.0, 5.0, 14.5, 35.0],
 [2009.0, 5.0, 15.5, 18.0],
 [2009.0, 5.0, 16.5, 37.0],
 [2009.0, 5.0, 17.5, 17.0],
 [2009.0, 5.0, 18.5, 46.0],
 [2009.0, 5.0, 19.5, 12.0],
 [2009.0, 5.0, 23.5, 40.0],
 [2009.0, 5.0, 31.5, 40.0],
 [2009.0, 6.0, 1.5, 122.0],
 [2009.0, 6.0, 2.5, 101.0],
 [2009.0, 6.0, 3.5, 17.0],
 [2009.0, 6.0, 4.5, 35.0],
 [2009.0, 6.0, 5.5, 16.0],
 [2009.0, 6.0, 8.5, 36.0],
 [2009.0, 6.0, 9.5, 36.0],
 [2009.0, 6.0, 12.5, 12.0],
 [2009.0, 6.0, 17.5, 6.0],
 [2009.0, 6.0, 21.5, 33.0],
 [20